# Create a Local Ethernet (Layer 2) Network

This notebook shows how to create an isolated local Ethernet and connect compute nodes to it.  


## Import the FABlib Library


In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

-----------------------------------  -------------------------------------------------
credmgr_host                         cm.fabric-testbed.net
orchestrator_host                    orchestrator.fabric-testbed.net
fabric_token                         /home/fabric/work/fabric_config/fabric_token.json
project_id                           1630021f-0a0c-4792-a241-997f410d36e1
bastion_username                     minawm_0041350787
bastion_key_filename                 /home/fabric/work/Mina_Bastion_Key2
bastion_public_addr                  bastion-1.fabric-testbed.net
bastion_passphrase                   None
slice_public_key_file                /home/fabric/.ssh/id_rsa.pub
slice_private_key_file               /home/fabric/.ssh/id_rsa
fabric_slice_private_key_passphrase  None
fablib_log_file                      /tmp/fablib/fablib.log
fablib_log_level                     INFO
-----------------------------------  -------------------------------------------------


## (Optional): Query for Available Testbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [2]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
MICH         6  192/192  1536/1536  60600/60600    381/381                 2/2                             2/2                            10/10               2/2               3/3
UTAH        10  316/320  2544/2560  116380/116400  634/635                 2/2                             4/4                            16/16               4/4               5/5
TACC        10  270/320  2478/2560  116210/116400  625/635                 0/2                             2/4                            16/16               4/4               6/6
WASH         6  192/192  1536/1536  60600/60600    381/381                 2/2

## Create the Experiment Slice

The following creates two nodes with basic NICs connected to an isolated local Ethernet.  

Two nodes are created and one NIC component is added to each node.  This example uses components of model `NIC_Basic` which are SR-IOV Virtual Function on a 100 Gpbs Mellanox ConnectX-6 PCI device. The VF is accessed by the node via PCI passthrough. Other NIC models are listed below. When using dedicated PCI devices the whole physical device is allocated to one node and the device is accessed by the node using PCI passthrough. Calling the `get_interfaces()` method on a component will return a list of interfaces. Many dedicated NIC components may have more than one port.  Either port can be connected to the network.

Next, add an `l2network` to the slice and pass the list of interfaces you want connected to this Ethernet. If all interfaces in the list are located on the same site, the network will automatically be a local Ethernet.  By default, a node is put on a random site.  If you want to ensure that your nodes are all on the same site you can specify the name of the site in the `add_node` methode.  You can use the `fablib.get_random_site()` method to get a random site name that can be used for both nodes.

NIC component models options:
- NIC_Basic: 100 Gbps Mellanox ConnectX-6 SR-IOV VF (1 Port)
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports) 
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports) 

In [60]:
slice_name = 'MySlice1'

site = 'MAX' #fablib.get_random_site()
print(f"Site: {site}")

node1_name = 'Node1'
node2_name = 'Node2'
network_name='net1'
node1_nic_name = 'nic1'
node2_nic_name = 'nic2'
image='default_ubuntu_20'

cores = 32
ram = 128
disk = 10

Site: MAX


In [61]:
try:
    #Create Slice
    slice4 = fablib.new_slice(name=slice_name)

    # Node1
    node1 = slice4.add_node(name=node1_name, site=site, cores=cores, ram=ram, disk=disk)
    iface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]
    node1.set_image(image)
    
    # Node2
    node2 = slice4.add_node(name=node2_name, site=site, cores=cores, ram=ram, disk=disk)
    iface2 = node2.add_component(model='NIC_Basic', name=node2_nic_name).get_interfaces()[0]
    node2.set_image(image)
    
    # Network
    net1 = slice4.add_l2network(name=network_name, interfaces=[iface1, iface2])

    #Submit Slice Request
    slice4.submit()
except Exception as e:
    print(f"Exception: {e}")


---------------  ------------------------------------
Slice Name       MySlice1
Slice ID         6b427c19-1aef-4302-9fe1-8a8e143df119
Slice State      StableOK
Lease End (UTC)  2022-07-21 12:50:17 +0000
---------------  ------------------------------------

Retry: 19, Time: 208 sec

ID                                    Name    Site    Host                         Cores    RAM    Disk  Image              Management IP    State    Error
------------------------------------  ------  ------  -------------------------  -------  -----  ------  -----------------  ---------------  -------  -------
aadfc032-4482-4ed2-ae12-cd3a6b554043  Node1   MAX     max-w4.fabric-testbed.net       32    128      10  default_ubuntu_20  63.239.135.84    Active
181de3af-789c-4a7a-857a-c36d1dfc9d34  Node2   MAX     max-w5.fabric-testbed.net       32    128      10  default_ubuntu_20  63.239.135.95    Active

Time to stable 208 seconds
Running post_boot_config ... Time to post boot config 214 seconds

Name      

## Observe the Slice's Attributes

### Print the slice 

In [62]:
try:
    slice4 = fablib.get_slice(name=slice_name)
    print(f"{slice4}")
except Exception as e:
    print(f"Exception: {e}")

---------------  ------------------------------------
Slice Name       MySlice1
Slice ID         6b427c19-1aef-4302-9fe1-8a8e143df119
Slice State      StableOK
Lease End (UTC)  2022-07-21 12:50:17 +0000
---------------  ------------------------------------


## Print the Node List

In [63]:
try:
    slice4 = fablib.get_slice(name=slice_name)

    print(f"{slice4.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

ID                                    Name    Site    Host                         Cores    RAM    Disk  Image              Management IP    State    Error
------------------------------------  ------  ------  -------------------------  -------  -----  ------  -----------------  ---------------  -------  -------
aadfc032-4482-4ed2-ae12-cd3a6b554043  Node1   MAX     max-w4.fabric-testbed.net       32    128      10  default_ubuntu_20  63.239.135.84    Active
181de3af-789c-4a7a-857a-c36d1dfc9d34  Node2   MAX     max-w5.fabric-testbed.net       32    128      10  default_ubuntu_20  63.239.135.95    Active


## Print the Node Details

In [64]:
try:
    slice4 = fablib.get_slice(name=slice_name)
    for node in slice4.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

-----------------  ------------------------------------------------------------------------------------------------------
ID                 aadfc032-4482-4ed2-ae12-cd3a6b554043
Name               Node1
Cores              32
RAM                128
Disk               10
Image              default_ubuntu_20
Image Type         qcow2
Host               max-w4.fabric-testbed.net
Site               MAX
Management IP      63.239.135.84
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/.ssh/id_rsa -J minawm_0041350787@bastion-1.fabric-testbed.net ubuntu@63.239.135.84
-----------------  ------------------------------------------------------------------------------------------------------
-----------------  ------------------------------------------------------------------------------------------------------
ID                 181de3af-789c-4a7a-857a-c36d1dfc9d34
Name               Node2
Cores              32
RAM                128
Disk               10
Image        

## Print the Interfaces

In [65]:
try:
    slice4 = fablib.get_slice(name=slice_name)
    
    print(f"{slice4.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

Name           Node    Network      Bandwidth  VLAN    MAC                Physical OS Interface    OS Interface
-------------  ------  ---------  -----------  ------  -----------------  -----------------------  --------------
Node1-nic1-p1  Node1   net1                 0          16:F5:89:97:9C:62  ens7                     ens7
Node2-nic2-p1  Node2   net1                 0          02:1A:9D:93:82:A9  ens7                     ens7


##  Configure IP Addresses

Some experiments use FABRIC layer 2 networks to enable deploying non-IP layer 3 networks.  If this describes your experiment, your nodes and network are ready. You can now login to the nodes and deploy your experiment.

Most users will want to configure IP addresses on there new nodes.  FABlib provides some useful methods to help you configure basic IP addresses. 

### Pick a Subnet

Create a subnet and list of available IP addresses. All objects are Python IP management objects. You can use either IPv4 or IPv6 subnets and addresses.

In [66]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet = IPv4Network("192.168.1.0/24")
    available_ips = list(subnet)[1:]
except Exception as e:
    print(f"Exception: {e}")

### Configure Node1

Get the node and the interface you wish to configure.  You can use `node.get_interface` to get the interface that is connected to the specified network.  Then `pop` an IP address from the list of available IPs and call `iface.ip_addr_add` to set the IP and subnet.  

Optionally, use the `node.execute()` method to show the results of adding the IP address.

In [67]:
try:
    node1 = slice4.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network_name) 
    node1_addr = available_ips.pop(0)
    node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet)
    
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 16:f5:89:97:9c:62 brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.1/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::14f5:89ff:fe97:9c62/64 scope link 
       valid_lft forever preferred_lft forever



### Configure Node2

Repeat the steps to add the next available IP to the second node.

In [68]:
try:
    node2 = slice4.get_node(name=node2_name)        
    node2_iface = node2.get_interface(network_name=network_name)  
    node2_addr = available_ips.pop(0)
    node2_iface.ip_addr_add(addr=node2_addr, subnet=subnet)
    
    stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_os_interface()}')
    print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:1a:9d:93:82:a9 brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.2/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::1a:9dff:fe93:82a9/64 scope link 
       valid_lft forever preferred_lft forever



## Run the Experiment

We will find the ping round trip time for this pair of sites.  Your experiment should be more interesting!


In [69]:
try:
#     node1 = slice4.get_node(name=node1_name)        

    stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Exception: {e}")

PING 192.168.1.2 (192.168.1.2) 56(84) bytes of data.
64 bytes from 192.168.1.2: icmp_seq=1 ttl=64 time=0.336 ms
64 bytes from 192.168.1.2: icmp_seq=2 ttl=64 time=0.096 ms
64 bytes from 192.168.1.2: icmp_seq=3 ttl=64 time=0.067 ms
64 bytes from 192.168.1.2: icmp_seq=4 ttl=64 time=0.061 ms
64 bytes from 192.168.1.2: icmp_seq=5 ttl=64 time=0.053 ms

--- 192.168.1.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4089ms
rtt min/avg/max/mdev = 0.053/0.122/0.336/0.107 ms




---

## Installing iperf

In [70]:
stdout, stderr = node1.execute(f'sudo apt update && sudo apt install -y iperf iperf3')
print (stdout)
print (stderr)

stdout, stderr = node2.execute(f'sudo apt update && sudo apt install -y iperf iperf3')
print (stdout)
print (stderr)

Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe Translation-en [5124 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 c-n-f Metadata [265 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [144 kB]
Get:9 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse Translation-en [104 kB]
Get:10 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse amd64 c-n-f Metadata [9136 B]
Get:11 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [1989 kB]
Get:12 http://nova.clouds.archiv

## Setting Parameters

In [75]:
stdout, stderr = node1.execute('echo "net.core.rmem_max = 2147483647\nnet.core.wmem_max = 2147483647\nnet.ipv4.tcp_rmem = 4096 87380 2147483647\nnet.ipv4.tcp_wmem = 4096 65536 2147483647\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')
print (stdout)
print (stderr)

net.core.rmem_max = 2147483647
net.core.wmem_max = 2147483647
net.ipv4.tcp_rmem = 4096 87380 2147483647
net.ipv4.tcp_wmem = 4096 65536 2147483647
net.ipv4.tcp_congestion_control=htcp
net.ipv4.tcp_mtu_probing=1
net.core.default_qdisc = fq

net.core.rmem_max = 2147483647
net.core.wmem_max = 2147483647
net.ipv4.tcp_rmem = 4096 87380 2147483647
net.ipv4.tcp_wmem = 4096 65536 2147483647
net.ipv4.tcp_congestion_control = htcp
net.ipv4.tcp_mtu_probing = 1
net.core.default_qdisc = fq




In [76]:
stdout, stderr = node2.execute('echo "net.core.rmem_max = 2147483647\nnet.core.wmem_max = 2147483647\nnet.ipv4.tcp_rmem = 4096 87380 2147483647\nnet.ipv4.tcp_wmem = 4096 65536 2147483647\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')
print (stdout)
print (stderr)

net.core.rmem_max = 2147483647
net.core.wmem_max = 2147483647
net.ipv4.tcp_rmem = 4096 87380 2147483647
net.ipv4.tcp_wmem = 4096 65536 2147483647
net.ipv4.tcp_congestion_control=htcp
net.ipv4.tcp_mtu_probing=1
net.core.default_qdisc = fq

net.core.rmem_max = 2147483647
net.core.wmem_max = 2147483647
net.ipv4.tcp_rmem = 4096 87380 2147483647
net.ipv4.tcp_wmem = 4096 65536 2147483647
net.ipv4.tcp_congestion_control = htcp
net.ipv4.tcp_mtu_probing = 1
net.core.default_qdisc = fq




In [79]:
stdout, stderr = node1.execute('tmux new -d \'iperf -s -f K\'')
print (stdout)
print (stderr)

In [90]:
stdout, stderr = node2.execute(f'iperf -c 192.168.1.1 -P 32 -w 999M') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 192.168.1.1, TCP port 5001
TCP window size: 1.95 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 34] local 192.168.1.2 port 41556 connected with 192.168.1.1 port 5001
[ 31] local 192.168.1.2 port 41552 connected with 192.168.1.1 port 5001
[ 32] local 192.168.1.2 port 41550 connected with 192.168.1.1 port 5001
[ 35] local 192.168.1.2 port 41554 connected with 192.168.1.1 port 5001
[  5] local 192.168.1.2 port 41498 connected with 192.168.1.1 port 5001
[  4] local 192.168.1.2 port 41496 connected with 192.168.1.1 port 5001
[  3] local 192.168.1.2 port 41494 connected with 192.168.1.1 port 5001
[  6] local 192.168.1.2 port 41500 connected with 192.168.1.1 port 5001
[  7] local 192.168.1.2 port 41502 connected with 192.168.1.1 port 5001
[  9] local 192.168.1.2 port 41504 connected with 192.168.1.1 port 5001
[ 12] local 192.168.1.2 port 41506 connected with 

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [59]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")